In [101]:
from q_value_calc_crosslinks import calcQ
from idXML2df_all_hits import readAndProcessIdXMLAllHits
from functions import get_target_id
import re

In [102]:
i = 2
dataset = {1: { 'type': 'crosslink_data',
                'name':'AChernev_080219_HeLa_RNA_UV',
                'comparison':'opti_'},
           2: { 'type': 'crosslink_data',
                'name':'M_Raabe_A_Wulf_220421_270421_Expl3_Ecoli_XL_UV_S30_LB_bRPfrac_11',
                'comparison':'perc'},
           3: { 'type': 'crosslink_data',
                'name':'M_Raabe_A_Wulf_220421_290421_Expl3_Ecoli_XL_DEB_S30_LB_bRPfrac_12',
                'comparison':'perc'},
           4: { 'type': 'crosslink_data',
                'name':'MRaabe_LW_091221_171221_Expl2_XL_Ecoli_NM_S30_bRP_rep1_11',
                'comparison':'perc'}}

In [103]:
input_files = [f"../data/{dataset[i]['type']}/{dataset[i]['name']}.idXML", 
      f"../data/{dataset[i]['type']}/{dataset[i]['name']}_{dataset[i]['comparison']}.idXML"]

In [104]:
for input_file in input_files:
    df = readAndProcessIdXMLAllHits(input_file)
    output_file = re.sub('.idXML', '.pkl', input_file)
    df.to_pickle(output_file)
    # filter dataframes
    filter_col = 'PSMId'
    filter_val = 1
    df = df.loc[df[filter_col] == filter_val,:]
    # compute q-values
    q_vals = calcQ(df)
    # compute target IDs
    target_id = get_target_id(q_vals)
    output_file = re.sub('.idXML', '_filtered.pkl', input_file)
    target_id.to_pickle(output_file)

c:\Users\mail\Research-Project\src\q_value_calc_crosslinks.py:100: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[labelColName].replace(to_replace=-1, value=0, inplace=True)
c:\Users\mail\Research-Project\src\q_value_calc_crosslinks.py:100: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a 